轉化PoH 類似Linkedlist : Sender|PoH Value|Receiver

In [2]:
import os
import time
import hashlib
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id, concat_ws, udf, lag, lead
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# ====== Java/Hadoop 路徑設定 ======
os.environ["JAVA_HOME"] = r"C:\Program Files\Gephi-0.10.1\jre-x64\jdk-11.0.17+8-jre"
os.environ["HADOOP_HOME"] = r"C:\Winutils"
os.environ["PATH"] = os.environ["JAVA_HOME"] + r";" + os.environ["HADOOP_HOME"] + r"\bin;" + os.environ["PATH"]

# ====== SparkSession ======
spark = SparkSession.builder \
    .appName("PoH LinkedList & Edge Construction") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()
spark.sparkContext.setCheckpointDir("file:///C:/Users/Leon/Desktop/spark-checkpoint")

# ====== 開始計時 ======
start_time = time.time()

# ====== 讀取資料 ======
raw_path = r"C:\Users\Leon\Desktop\程式語言資料\python\TD-UF\Anti Money Laundering Transaction Data (SAML-D)\SAML-D.csv"
out_dir = r"C:\Users\Leon\Desktop\程式語言資料\python\TD-UF\圖結構"
os.makedirs(out_dir, exist_ok=True)

df = spark.read.csv(raw_path, header=True, inferSchema=True)
df = df.withColumn("tx_id", monotonically_increasing_id())

# ====== 轉成 timestamp & 排序 ======
def to_epoch(dt):
    try:
        return int(time.mktime(time.strptime(dt, "%Y-%m-%d %H:%M:%S")))
    except Exception:
        return 0

udf_epoch = udf(to_epoch, IntegerType())
df = df.withColumn("timestamp", udf_epoch(concat_ws(" ", col("Date").cast(StringType()), col("Time").cast(StringType()))))
df = df.orderBy("timestamp")

# ====== row_idx 產生 PoH LinkedList ======
w = Window.orderBy("timestamp")
df = df.withColumn("row_idx", row_number().over(w) - 1)

# ====== 收集必要欄位成 Pandas DataFrame，計算 PoH 串鍊 ======
df_pd = df.select("row_idx", "Sender_account", "Receiver_account", "tx_id", "timestamp").orderBy("row_idx").toPandas()

pohs = []
prev_poh = ""
for _, row in df_pd.iterrows():
    sender = str(row['Sender_account'])
    receiver = str(row['Receiver_account'])
    txid = str(row['tx_id'])
    poh_input = prev_poh + sender + receiver + txid
    poh = hashlib.sha256(poh_input.encode()).hexdigest()
    pohs.append(poh)
    prev_poh = poh

df_pd["PoH_Value"] = pohs

# ====== 生成 Spark 節點表 DataFrame ======
node_pd = df_pd[["Sender_account", "PoH_Value", "Receiver_account", "timestamp"]]
nodes_path = os.path.join(out_dir, "nodes_full.csv")
node_pd.to_csv(nodes_path, index=False, encoding="utf-8-sig")
print(f"✔️ 節點表輸出完成：{nodes_path}")

# ====== 建 Receiver→Sender 有向邊表（找每個帳戶收到錢→首次轉出） ======
# 收到錢的所有紀錄
receiver_rows = {}
for idx, row in df_pd.iterrows():
    receiver = row['Receiver_account']
    receiver_rows.setdefault(receiver, []).append(idx)

# 首次出現當Sender的紀錄
sender_first_rows = {}
for idx, row in df_pd.iterrows():
    sender = row['Sender_account']
    if sender not in sender_first_rows:
        sender_first_rows[sender] = idx

edges = []
for acct, recv_idxs in receiver_rows.items():
    if acct in sender_first_rows:
        send_idx = sender_first_rows[acct]
        # 若第一次出現不是同一筆，才建邊
        for recv_idx in recv_idxs:
            if recv_idx < send_idx:
                src = df_pd.loc[recv_idx, 'PoH_Value']
                tgt = df_pd.loc[send_idx, 'PoH_Value']
                edges.append([src, tgt, acct])

edges_path = os.path.join(out_dir, "edges_full.csv")
import pandas as pd
edges_df = pd.DataFrame(edges, columns=['from_PoH', 'to_PoH', 'account'])
edges_df.to_csv(edges_path, index=False, encoding="utf-8-sig")
print(f"✔️ 有向邊表輸出完成：{edges_path}")

# ====== 統計運算時間 ======
end_time = time.time()
print(f"✨ 完成所有金流圖構建，總花費時間：{end_time - start_time:.2f} 秒")

spark.stop()


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [1]:
import os
import time
import hashlib
from tqdm import tqdm
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, concat_ws, monotonically_increasing_id, row_number
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
from graphframes import GraphFrame
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression as SparkLR
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from tabulate import tabulate
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# ====== Java/Hadoop 路徑設定 ======
os.environ["JAVA_HOME"] = r"C:\Program Files\Gephi-0.10.1\jre-x64\jdk-11.0.17+8-jre"
os.environ["HADOOP_HOME"] = r"C:\Winutils"
os.environ["PATH"] = os.environ["JAVA_HOME"] + r";" + os.environ["HADOOP_HOME"] + r"\bin;" + os.environ["PATH"]

# ====== 建立 SparkSession ======
print("🚀 啟動 SparkSession...")
spark = SparkSession.builder \
    .appName("TD-UF vs Baseline, PySpark LR, Sliding Window 5-Fold") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.1-s_2.12") \
    .getOrCreate()
spark.sparkContext.setCheckpointDir("file:///C:/Users/Leon/Desktop/spark-checkpoint")
print("✔️ SparkSession 建立完成")

# ====== 日期字串轉 epoch timestamp ======
def to_epoch(dt):
    try:
        return int(time.mktime(time.strptime(dt, "%Y-%m-%d %H:%M:%S")))
    except Exception:
        return 0
udf_epoch = udf(to_epoch, IntegerType())

# ====== 讀取資料 ======
print("📥 讀取資料中 ...")
raw_path = r"C:\Users\Leon\Desktop\程式語言資料\python\TD-UF\Anti Money Laundering Transaction Data (SAML-D)\SAML-D.csv"
df = spark.read.csv(raw_path, header=True, inferSchema=True)
df = df.withColumn("timestamp", udf_epoch(concat_ws(" ", col("Date").cast("string"), col("Time").cast("string"))))
df = df.withColumn("tx_id", monotonically_increasing_id())
print(f"✔️ 載入資料完成，共 {df.count()} 筆交易")

# ====== PoH 鏈式雜湊產生 ======
def poh_chain(records):
    pohs = []
    prev_poh = ""
    for row in tqdm(records, desc="🔗 產生 PoH 雜湊鏈中..."):
        txid_str = str(row['tx_id'])
        poh = hashlib.sha256((prev_poh + txid_str).encode()).hexdigest()
        pohs.append(poh)
        prev_poh = poh
    return pohs

print("🔄 產生 PoH 鏈式雜湊欄位 ...")
tx_pd = df.orderBy("tx_id").select(
    "tx_id", "Sender_account", "Receiver_account", "timestamp", "Amount", "Is_laundering"
).toPandas()
tx_pd["poh"] = poh_chain(tx_pd.to_dict('records'))
df = spark.createDataFrame(tx_pd)
print("✔️ PoH 雜湊計算完成")

# ====== 交易圖建構：每筆交易為節點 ======
print("🧩 建立交易圖（每筆交易為一個節點）...")
edges_df = df.alias("a").join(
    df.alias("b"),
    (col("a.Receiver_account") == col("b.Sender_account")) &
    (col("a.timestamp") < col("b.timestamp")),
    "inner"
).select(
    col("a.tx_id").alias("src"),
    col("b.tx_id").alias("dst")
).distinct()
vertices = df.select(col("tx_id").alias("id")).distinct()
print(f"✔️ 交易節點數: {vertices.count()}，交易邊數: {edges_df.count()}")

print("🔗 TD-UF 分群（每筆交易都有 component id）...")
gf = GraphFrame(vertices, edges_df)
components = gf.connectedComponents()
print(f"✔️ 完成分群計算")
df = df.join(components, df.tx_id == components.id, "left").drop("id")
df = df.withColumn("component", col("component").cast("double"))

# ====== Baseline 特徵工程（含編碼）======
df_baseline = df
categorical_cols = ["Payment_currency", "Received_currency", "Sender_bank_location", "Receiver_bank_location", "Payment_type"]
for c in categorical_cols:
    indexer = StringIndexer(inputCol=c, outputCol=f"{c}_idx").fit(df_baseline)
    df_baseline = indexer.transform(df_baseline)

feature_cols_baseline = ["Amount", "timestamp"] + [f"{c}_idx" for c in categorical_cols]
feature_cols_tduf = ["Amount", "timestamp", "component"]

# ========== 加入時間排序 row_id ==========
window = Window.orderBy("timestamp")
df_baseline = df_baseline.withColumn("row_id", row_number().over(window) - 1)
df = df_baseline  # baseline和tduf保證排序相同
total_count = df.count()
K = 5
fold_size = total_count // (K+1)   # 注意sliding window預留測試區

# ====== Sliding window 設計：每次擴展訓練集，測試集為下一區間 ======
def sliding_window_splits(total_count, fold_size, K):
    splits = []
    for k in range(K):
        train_end = (k+1) * fold_size
        test_start = train_end
        test_end = test_start + fold_size
        if test_end > total_count:
            test_end = total_count
        splits.append((0, train_end, test_start, test_end))
    return splits

splits = sliding_window_splits(total_count, fold_size, K)

evaluator = MulticlassClassificationEvaluator(labelCol="Is_laundering", predictionCol="prediction")

# ========== TD-UF+LR sliding window 5-fold ==========
print("\n==== ⏳ TD-UF+LR 滑動視窗五折 ====")
metrics_tduf, all_true_tduf, all_pred_tduf, all_proba_tduf = [], [], [], []
for i, (train_start, train_end, test_start, test_end) in enumerate(splits):
    train_df = df.filter((col("row_id") >= train_start) & (col("row_id") < train_end))
    test_df  = df.filter((col("row_id") >= test_start) & (col("row_id") < test_end))
    n_train_pos = train_df.filter(col("Is_laundering") == 1).count()
    n_test_pos = test_df.filter(col("Is_laundering") == 1).count()
    print(f"[TD-UF] Fold {i+1}: 訓練集({train_end-train_start})正樣本={n_train_pos}，測試集({test_end-test_start})正樣本={n_test_pos}")

    assembler = VectorAssembler(inputCols=feature_cols_tduf, outputCol="features")
    train_data = assembler.transform(train_df).select("features", "Is_laundering")
    test_data  = assembler.transform(test_df).select("features", "Is_laundering")

    lr = SparkLR(maxIter=100, regParam=0.01, labelCol="Is_laundering", probabilityCol="probability")
    t1 = time.time()
    model = lr.fit(train_data)
    pred = model.transform(test_data)
    t2 = time.time()

    acc = evaluator.setMetricName("accuracy").evaluate(pred)
    prec = evaluator.setMetricName("weightedPrecision").evaluate(pred)
    rec = evaluator.setMetricName("weightedRecall").evaluate(pred)
    f1 = evaluator.setMetricName("f1").evaluate(pred)
    metrics_tduf.append((acc, prec, rec, f1, t2-t1))

    pred_pd = pred.select("Is_laundering", "prediction", "probability").toPandas()
    all_true_tduf.append(pred_pd["Is_laundering"].values)
    all_pred_tduf.append(pred_pd["prediction"].values)
    all_proba_tduf.append(pred_pd["probability"].apply(lambda x: x[1] if hasattr(x, "__getitem__") else x).values)

y_true_tduf = np.concatenate(all_true_tduf)
y_pred_tduf = np.concatenate(all_pred_tduf)
y_proba_tduf = np.concatenate(all_proba_tduf)
metrics_tduf = np.array(metrics_tduf)

print("\n📌 TD-UF+LR（sliding window 五折驗證）")
print(tabulate([
    ("Accuracy", np.mean(metrics_tduf[:,0])), 
    ("Precision", np.mean(metrics_tduf[:,1])),
    ("Recall", np.mean(metrics_tduf[:,2])),
    ("F1 Score", np.mean(metrics_tduf[:,3])),
    ("Avg Time", np.mean(metrics_tduf[:,4]))
], headers=["指標", "平均值"], tablefmt="fancy_grid"))

# ====== 畫出 TD-UF+LR 的 ROC 曲線 ======
fpr, tpr, _ = roc_curve(y_true_tduf, y_proba_tduf)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, label=f'TD-UF+LR (AUC = {roc_auc:.4f})')

# ====== Baseline sliding window 5-fold（同一組分割）=====
print("\n==== ⏳ Baseline LR sliding window 五折 ====")
metrics_base, all_true_base, all_pred_base, all_proba_base = [], [], [], []
for i, (train_start, train_end, test_start, test_end) in enumerate(splits):
    train_df = df.filter((col("row_id") >= train_start) & (col("row_id") < train_end))
    test_df  = df.filter((col("row_id") >= test_start) & (col("row_id") < test_end))
    n_train_pos = train_df.filter(col("Is_laundering") == 1).count()
    n_test_pos = test_df.filter(col("Is_laundering") == 1).count()
    print(f"[Baseline] Fold {i+1}: 訓練集({train_end-train_start})正樣本={n_train_pos}，測試集({test_end-test_start})正樣本={n_test_pos}")

    assembler = VectorAssembler(inputCols=feature_cols_baseline, outputCol="features")
    train_data = assembler.transform(train_df).select("features", "Is_laundering")
    test_data  = assembler.transform(test_df).select("features", "Is_laundering")

    lr = SparkLR(maxIter=100, regParam=0.01, labelCol="Is_laundering", probabilityCol="probability")
    t1 = time.time()
    model = lr.fit(train_data)
    pred = model.transform(test_data)
    t2 = time.time()

    acc = evaluator.setMetricName("accuracy").evaluate(pred)
    prec = evaluator.setMetricName("weightedPrecision").evaluate(pred)
    rec = evaluator.setMetricName("weightedRecall").evaluate(pred)
    f1 = evaluator.setMetricName("f1").evaluate(pred)
    metrics_base.append((acc, prec, rec, f1, t2-t1))

    pred_pd = pred.select("Is_laundering", "prediction", "probability").toPandas()
    all_true_base.append(pred_pd["Is_laundering"].values)
    all_pred_base.append(pred_pd["prediction"].values)
    all_proba_base.append(pred_pd["probability"].apply(lambda x: x[1] if hasattr(x, "__getitem__") else x).values)

y_true_base = np.concatenate(all_true_base)
y_pred_base = np.concatenate(all_pred_base)
y_proba_base = np.concatenate(all_proba_base)
metrics_base = np.array(metrics_base)

print("\n📌 Baseline LR（sliding window 五折驗證）")
print(tabulate([
    ("Accuracy", np.mean(metrics_base[:,0])), 
    ("Precision", np.mean(metrics_base[:,1])),
    ("Recall", np.mean(metrics_base[:,2])),
    ("F1 Score", np.mean(metrics_base[:,3])),
    ("Avg Time", np.mean(metrics_base[:,4]))
], headers=["指標", "平均值"], tablefmt="fancy_grid"))

# ====== 畫出 Baseline LR 的 ROC 曲線 ======
fpr_b, tpr_b, _ = roc_curve(y_true_base, y_proba_base)
roc_auc_b = auc(fpr_b, tpr_b)
plt.plot(fpr_b, tpr_b, label=f'Baseline LR (AUC = {roc_auc_b:.4f})')

plt.plot([0,1], [0,1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve: TD-UF+LR vs Baseline (Sliding Window 5-Fold)")
plt.legend()
plt.grid(True, axis="both")
plt.tight_layout()
plt.show()

spark.stop()
print("✅ 全部流程執行完畢！")


🚀 啟動 SparkSession...
✔️ SparkSession 建立完成
📥 讀取資料中 ...
✔️ 載入資料完成，共 9504852 筆交易
🔄 產生 PoH 鏈式雜湊欄位 ...


🔗 產生 PoH 雜湊鏈中...: 100%|██████████| 9504852/9504852 [00:11<00:00, 793376.74it/s]


✔️ PoH 雜湊計算完成
🧩 建立交易圖（每筆交易為一個節點）...
✔️ 交易節點數: 9504852，交易邊數: 0
🔗 TD-UF 分群（每筆交易都有 component id）...


c:\Users\Leon\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
c:\Users\Leon\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


✔️ 完成分群計算


Py4JJavaError: An error occurred while calling o161.fit.
: org.apache.spark.SparkException: Input column Payment_currency does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	a